In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests6

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2

def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


# a  = .25, k= 4, something not right?

In [24]:
a = 0.25
num_params=4
num_sims = 100

In [25]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.97 & 0.96 & 1.00 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.03 & 0.03 & 0.04 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.97, 0.  , 0.03]), array([0.97, 0.  , 0.03]), array([0.96, 0.  , 0.04]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), -2.249267922748295, 5.2393809035209395, 5.723408698574368)


In [26]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.97 & 0.97 & 0.98 & 0.98 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.01 & 0.01 & 0.00   \\
Model 2 & 0.03 & 0.03 & 0.03 & 0.01 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.97, 0.  , 0.03]), array([0.97, 0.  , 0.03]), array([0.97, 0.  , 0.03]), array([0.98, 0.01, 0.01]), array([0.98, 0.01, 0.01]), array([0.99, 0.  , 0.01]), -2.5471521184182824, 7.478193080535721, 7.734398536438539)


# a  = .25, k= 9

In [3]:
a = 0.25
num_params=9
num_sims = 100

In [4]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.92 & 0.92 & 0.91 & 0.98 & 0.98 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.01 & 0.01 & 0.00   \\
Model 2 & 0.08 & 0.08 & 0.09 & 0.01 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.92, 0.  , 0.08]), array([0.92, 0.  , 0.08]), array([0.91, 0.  , 0.09]), array([0.98, 0.01, 0.01]), array([0.98, 0.01, 0.01]), array([0.99, 0.  , 0.01]), -4.172891874535366, 5.843317865377202, 5.853982179185028)


In [29]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.98 & 0.98 & 0.95 & 0.98 & 0.98 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.01 & 0.01 & 0.01 & 0.01   \\
Model 2 & 0.02 & 0.02 & 0.04 & 0.01 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.98, 0.  , 0.02]), array([0.98, 0.  , 0.02]), array([0.95, 0.01, 0.04]), array([0.98, 0.01, 0.01]), array([0.98, 0.01, 0.01]), array([0.98, 0.01, 0.01]), -3.903703712222221, 6.291603787966217, 7.887010178631539)


In [30]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=100, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.89 & 0.96 & 0.91 & 0.97 & 0.97 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.02 & 0.02 & 0.01   \\
Model 2 & 0.11 & 0.04 & 0.09 & 0.01 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.89, 0.  , 0.11]), array([0.96, 0.  , 0.04]), array([0.91, 0.  , 0.09]), array([0.97, 0.02, 0.01]), array([0.97, 0.02, 0.01]), array([0.98, 0.01, 0.01]), -4.410367699484034, 3.943549815067037, 4.475365964249693)


# a  = .25, K = 19

In [9]:
a = 0.25
num_params=19
num_sims = 100

In [18]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.77 & 0.88 & 0.79 & 1.00 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.23 & 0.12 & 0.21 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.77, 0.  , 0.23]), array([0.88, 0.  , 0.12]), array([0.79, 0.  , 0.21]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), -8.515402181131781, 5.221951312874477, 6.630862781174881)


In [19]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.77 & 0.77 & 0.79 & 0.97 & 0.97 & 0.97   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.01 & 0.01 & 0.01   \\
Model 2 & 0.23 & 0.23 & 0.21 & 0.02 & 0.02 & 0.02   \\
\hline
\end{tabular}
(array([0.77, 0.  , 0.23]), array([0.77, 0.  , 0.23]), array([0.79, 0.  , 0.21]), array([0.97, 0.01, 0.02]), array([0.97, 0.01, 0.02]), array([0.97, 0.01, 0.02]), -10.177877989647717, 7.622346191765633, 8.701770476544555)


# evidence of power

In [5]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9
num_sims = 100

def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [6]:
a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.83 & 0.83 & 0.82 & 0.53 & 0.53 & 0.60   \\
Model 1 & 0.17 & 0.17 & 0.18 & 0.47 & 0.47 & 0.40   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.83, 0.17, 0.  ]), array([0.83, 0.17, 0.  ]), array([0.82, 0.18, 0.  ]), array([0.53, 0.47, 0.  ]), array([0.53, 0.47, 0.  ]), array([0.6, 0.4, 0. ]), 5.722573224348203, 5.538740131191697, 5.252860376369723)


In [7]:
a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.82 & 0.82 & 0.84 & 0.53 & 0.53 & 0.60   \\
Model 1 & 0.18 & 0.18 & 0.16 & 0.47 & 0.47 & 0.40   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.82, 0.18, 0.  ]), array([0.82, 0.18, 0.  ]), array([0.84, 0.16, 0.  ]), array([0.53, 0.47, 0.  ]), array([0.53, 0.47, 0.  ]), array([0.6, 0.4, 0. ]), 6.761781369387777, 4.764567397987834, 5.518445814431548)


In [8]:
a1,a2 = np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=100, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.81 & 0.88 & 0.84 & 0.41 & 0.41 & 0.56   \\
Model 1 & 0.19 & 0.12 & 0.16 & 0.59 & 0.59 & 0.44   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.81, 0.19, 0.  ]), array([0.88, 0.12, 0.  ]), array([0.84, 0.16, 0.  ]), array([0.41, 0.59, 0.  ]), array([0.41, 0.59, 0.  ]), array([0.56, 0.44, 0.  ]), 6.30810682949157, 4.539392310327888, 5.019541618245677)


In [9]:
a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.53 & 0.53 & 0.54 & 0.36 & 0.36 & 0.48   \\
Model 1 & 0.47 & 0.47 & 0.46 & 0.64 & 0.64 & 0.52   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.53, 0.47, 0.  ]), array([0.53, 0.47, 0.  ]), array([0.54, 0.46, 0.  ]), array([0.36, 0.64, 0.  ]), array([0.36, 0.64, 0.  ]), array([0.48, 0.52, 0.  ]), 9.162089665540336, 4.9080146208080135, 4.857706552860274)


In [10]:
a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.80 & 0.80 & 0.81 & 0.46 & 0.46 & 0.54   \\
Model 1 & 0.20 & 0.20 & 0.19 & 0.54 & 0.54 & 0.46   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.8, 0.2, 0. ]), array([0.8, 0.2, 0. ]), array([0.81, 0.19, 0.  ]), array([0.46, 0.54, 0.  ]), array([0.46, 0.54, 0.  ]), array([0.54, 0.46, 0.  ]), 6.66768832111772, 4.897240400562319, 5.387975141125211)


# evidence of power 2

In [11]:
a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.08 & 0.16 & 0.07 & 0.39 & 0.39 & 0.44   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.92 & 0.84 & 0.93 & 0.61 & 0.61 & 0.56   \\
\hline
\end{tabular}
(array([0.08, 0.  , 0.92]), array([0.16, 0.  , 0.84]), array([0.07, 0.  , 0.93]), array([0.39, 0.  , 0.61]), array([0.39, 0.  , 0.61]), array([0.44, 0.  , 0.56]), -15.423622463395306, 5.247142160813506, 5.401879865612109)


In [12]:
a2,a1 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.11 & 0.17 & 0.10 & 0.59 & 0.59 & 0.65   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.89 & 0.83 & 0.90 & 0.41 & 0.41 & 0.35   \\
\hline
\end{tabular}
(array([0.11, 0.  , 0.89]), array([0.17, 0.  , 0.83]), array([0.1, 0. , 0.9]), array([0.59, 0.  , 0.41]), array([0.59, 0.  , 0.41]), array([0.65, 0.  , 0.35]), -14.227971939243421, 5.134937697796379, 5.378442319659229)


In [13]:
a2,a1 =np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=100, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.09 & 0.53 & 0.08 & 0.45 & 0.45 & 0.58   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.91 & 0.47 & 0.92 & 0.55 & 0.55 & 0.42   \\
\hline
\end{tabular}
(array([0.09, 0.  , 0.91]), array([0.53, 0.  , 0.47]), array([0.08, 0.  , 0.92]), array([0.45, 0.  , 0.55]), array([0.45, 0.  , 0.55]), array([0.58, 0.  , 0.42]), -14.090738475842624, 4.786976953048194, 4.913199409069314)


In [14]:
a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.18 & 0.18 & 0.19 & 0.33 & 0.33 & 0.50   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.82 & 0.82 & 0.81 & 0.67 & 0.67 & 0.50   \\
\hline
\end{tabular}
(array([0.18, 0.  , 0.82]), array([0.18, 0.  , 0.82]), array([0.19, 0.  , 0.81]), array([0.33, 0.  , 0.67]), array([0.33, 0.  , 0.67]), array([0.5, 0. , 0.5]), -12.44122394987745, 4.400798466023099, 4.913313598953916)


In [15]:
a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.41 & 0.61 & 0.37 & 0.87 & 0.87 & 0.93   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.59 & 0.39 & 0.63 & 0.13 & 0.13 & 0.07   \\
\hline
\end{tabular}
(array([0.41, 0.  , 0.59]), array([0.61, 0.  , 0.39]), array([0.37, 0.  , 0.63]), array([0.87, 0.  , 0.13]), array([0.87, 0.  , 0.13]), array([0.93, 0.  , 0.07]), -9.676647933017739, 3.8902246680571846, 4.386896961918997)
